# Phytosensing – Using Plants as Sensors
## Introduction
In this tutorial we will develop a toolchain to analyse and classify plant electrical signals to infer the external stimulus of the given plant.
Because collecting the data would take too much time we will use a dataset that we collected earlier (in Lübeck). It is a real world dataset.
I planned the practical in the way that you will be guided step by step. I encourage the peoples who are familiar with programming in python (and the used libraries) to explore other possible approaches. In the end we will do a little challenge were you can implement a classifier (and possible pipeline) and the best performing classifier will win (maybe a prize maybe not).

## 1. Load the dataset (data acquisition)
Because we do not have to collect a dataset this section is only about loading data, visualization, and cutting the data into the appropriate experiment intervals. Hence we will do the following steps:

1. Load the dataset
2. Visualize the dataset
3. cut the dataset into the appropriate length

<em>Hint: for the final challenge you are allowed to use all available data (not only the cut data)</em>

### Import necessary libraries
These libraries are necessary for the tutorial. You can add more libraries if you want to use them later.

In [ ]:
# import the required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from scipy.signal import welch
from scipy.signal import resample
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer
# from keras.optimizers import Adam
from keras.optimizers.legacy import Adam
from sklearn import preprocessing
from numpy import genfromtxt

### Loading the dataset
The data were collected using the Cybres MU measurement system. Hence, we have a lot of information that are not interesting for us. We will only use the following columns: `timestamp`, `differential_potential_CH1`, and `differential_potential_CH2`. In a first step load the data into a pandas dataframe and print the first 5 rows.

In [ ]:
# load the data and print the first 5 rows
wind_1 = pd.read_csv('data/raw_data/wind/series_1.csv')
print(wind_1.head())

### reduce the dataset to the necessary information
As you see, the raw data contain much information we do not need (in this tutorial). So next step is to drop all but the relevant colums.

In [ ]:
# drop all but the relevant columns
wind_1 = wind_1[['timestamp', 'differential_potential_CH1', 'differential_potential_CH2']]
# reset the index
wind_1 = wind_1.reset_index(drop=True)

### Visualize the dataset
After you successfully removed unnecessary information, you can have a first look at the data. Please plot the two electrical signals over time.

In [ ]:
wind_1.plot(x='timestamp', y=['differential_potential_CH1', 'differential_potential_CH2'])

### Cut the dataset into the appropriate length
After plotting the data, we now want to cut them according to the experiments. The required timestamps are given in `experiment_start.csv` and `experiment_end.csv`. Load these files and cut the data into the appropriate intervals. For future processing you may want to save the intermediate results that you do not have to compute them again.

In [ ]:
stimulus = "temperature"
# load the experiment start timestamps
start_times = pd.read_csv(f"data/{stimulus}_experiment_start.csv", parse_dates=True)

running_exp_number = 0
for i in range(1, 5):
    # transform the timestamps to datetime
    start_times[f'series_{i}'] = pd.to_datetime(start_times[f'series_{i}'], format="%Y-%m-%d %H:%M:%S")
    # load data
    wind = pd.read_csv(f"data/raw_data/{stimulus}/series_{i}.csv")
    wind["timestamp"] = pd.to_datetime(wind["timestamp"], format="%Y-%m-%d %H:%M:%S")

    # iterate through data and cut
    for j in range(start_times.shape[0]):
        begin = start_times[f'series_{i}'].iloc[j]
        # because series have different amounts of experiments
        if begin is not pd.NaT:
            end = begin + timedelta(minutes=130)
            print(f"begin: {begin} -- end: {end}")
            tmp_wind = wind.loc[((wind["timestamp"] >= begin) & (wind["timestamp"] <= end))]

            tmp_wind = tmp_wind.reset_index(drop=True)

            tmp_wind = tmp_wind[['timestamp', 'differential_potential_CH1', 'differential_potential_CH2']]
            # plot
            plt.plot(tmp_wind["timestamp"], tmp_wind["differential_potential_CH1"], label="differential_potential_CH1")
            plt.plot(tmp_wind["timestamp"], tmp_wind["differential_potential_CH2"], label="differential_potential_CH2")
            plt.axvspan(tmp_wind["timestamp"].iloc[0]+timedelta(minutes=60), tmp_wind["timestamp"].iloc[0] + timedelta(minutes=70), facecolor='0.2', alpha=0.5)
            plt.legend()
            plt.show()
            # save back to csv
            tmp_wind.to_csv(f"data/cut_data/{stimulus}/experiment_{running_exp_number}.csv", index=False)
            running_exp_number += 1


### Cut data continued
Well done. By now you should have a folder of files, were every file contains one experiment. Now you have to repeat it for all stimuli and series. This can be done by using a for loop. If you are short on time you can skip this step and use the provided cut data (data/cut_data/*).

In [ ]:
# cut all the data
# ATTENTION if you want to cut the light data you have to use a different approach (because the experimental protocol is slightly different
# # blue light: starting at 1 am; every 6 hours
# 1 am, 7 am 1 pm, 7 pm
# red light: starting at 4 am; every 6 hours
# 4 am, 10 am, 4 pm, 10 pm

### Scaling
While cutting the data you probably recognised that the values are quite large. This is because they have not yet been scaled. Currently, the data are raw values from the measurement unit. You can convert them using the following formula:
$$ x \text{mV} = \frac{x_{\text{raw}} - 512000}{1000} $$

In [ ]:
# Scale the electric potential channels to mV
def scale_data(data):
    data['differential_potential_CH1'] = (data['differential_potential_CH1'] - 512000) / 1000
    data['differential_potential_CH2'] = (data['differential_potential_CH2'] - 512000) / 1000
    return data

for i in range(len(os.listdir("data/cut_data/wind"))):
    print(i)
    df = pd.read_csv(f"data/cut_data/wind/experiment_{i}.csv")
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")

    # scaling
    df = scale_data(df)

    # plot
    plt.plot(df["timestamp"], df["differential_potential_CH1"], label="differential_potential_CH1")
    plt.plot(df["timestamp"], df["differential_potential_CH2"], label="differential_potential_CH2")
    plt.axvspan(df["timestamp"].iloc[0]+timedelta(minutes=60), df["timestamp"].iloc[0] + timedelta(minutes=70), facecolor='0.2', alpha=0.5)
    plt.legend()
    plt.show()

## 2. Feature extraction
Now that we have the data in the correct format, we can start with the feature extraction. We begin by extracting features from the stimuli part and the initial 10 minutes for background subtraction. We will calculate the mean, std, and average power spectral density. (Attention some samples may contain NaN values/are incomplete!)

In [ ]:
def power_spectral_density(data):
    # Calculate PSD using welch estimate. PSD gives the spectra
    # depending on the frequencies. Sum over all spectra to
    # receive the total (average) spectral power
    _, psd = welch(data)  # TODO adjust parameters such as sampling frequency
    return sum(psd)


mean_ch1 = []
mean_ch2 = []
std_ch1 = []
std_ch2 = []
asp_ch1 = []
asp_ch2 = []

stimulus = "wind"

nb_experiments = len(os.listdir(f"data/scaled_data/{stimulus}"))

for i in range(nb_experiments):
    # print(f"loading experiment {i}")
    df = pd.read_csv(f"data/scaled_data/{stimulus}/experiment_{i}.csv")
    # deselect empty experiments
    if df.empty:
        print(f"experiment {i} is empty")
        continue

    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")

    # select stimulus phase and background subtraction phase
    stim = df[((df["timestamp"] >= df["timestamp"][0] + timedelta(minutes=60)) & (
            df["timestamp"] <= df["timestamp"][0] + timedelta(minutes=70)))]
    bg = df[((df["timestamp"] >= df["timestamp"][0] + timedelta(minutes=0)) & (
            df["timestamp"] <= df["timestamp"][0] + timedelta(minutes=10)))]

    # calculate features per channels
    # make sure the values can be calculated
    if stim["differential_potential_CH1"].mean() is np.NAN or bg["differential_potential_CH1"].mean() is np.NAN:
        print(f"experiment {i} has NaNs")
        continue
    # calculate mean
    mean_ch1.append(stim["differential_potential_CH1"].mean() - bg["differential_potential_CH1"].mean())
    mean_ch2.append(stim["differential_potential_CH2"].mean() - bg["differential_potential_CH2"].mean())

    # calculate std
    std_ch1.append(stim["differential_potential_CH1"].std() - bg["differential_potential_CH1"].std())
    std_ch2.append(stim["differential_potential_CH2"].std() - bg["differential_potential_CH2"].std())

    # calculate asp
    asp_ch1.append(power_spectral_density(stim["differential_potential_CH1"]) - power_spectral_density(bg["differential_potential_CH1"]))
    asp_ch2.append(power_spectral_density(stim["differential_potential_CH2"]) - power_spectral_density(bg["differential_potential_CH2"]))

df = pd.DataFrame({"mean_ch1": mean_ch1, "mean_ch2": mean_ch2, "std_ch1": std_ch1, "std_ch2": std_ch2, "asp_ch1": asp_ch1, "asp_ch2": asp_ch2})
df.to_csv(f"data/features/{stimulus}/mean_std_asp_features.csv", index=False)

### Feature extraction continued
I encourage you to find and implement other features.

In [ ]:
# TODO implement other interesting features

## 3. Data augmentation (Slicing)
This is voluntary. If you do not plan on using deep learning you can skip this part. If you just want to do deep learning, you can use just the raw time series. However, as you may notice, the dataset is rather small and probably not sufficient for deep learning. Hence, we can use data augmentation strategies to enlarge the dataset. A starting point can be the [tsaug library](url https://tsaug.readthedocs.io/en/stable/). It provides a lot of different data augmentation strategies for time series data. Some simple example is to add white noise.

In [ ]:
stimulus = "temperature"
n_experiments = len(os.listdir(f"data/scaled_data/{stimulus}"))

for i in range(n_experiments):
    # print(f"experiment {i}")
    df = pd.read_csv(f"data/scaled_data/{stimulus}/experiment_{i}.csv")
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S")

    if df.empty:
        print(f"skipping experiment {i} because of empty data")
        continue

    # get timings
    stimulus_application_begin = df["timestamp"].iloc[0] + timedelta(minutes=59)
    stimulus_application_end = df["timestamp"].iloc[0] + timedelta(minutes=60+11)  # 10 min stimulus

    # cut data
    df = df[df["timestamp"] >= stimulus_application_begin]
    df = df[df["timestamp"] <= stimulus_application_end]

    new_num_points = 418
    tmp_df = df.set_index('timestamp')
    if tmp_df.shape[0] < new_num_points-10:
        print(f"skipping experiment {i} because of too few data points")
        continue
    interpolated_df = resample(tmp_df, num=new_num_points)

    ch_1 = np.reshape(interpolated_df[:, 0], (1, -1))
    ch_2 = np.reshape(interpolated_df[:, 1], (1, -1))

    # TODO augment data! (e.g. add white noise)
    if i == 0:
        ch_1_all = ch_1
        ch_2_all = ch_2
    else:
        ch_1_all = np.vstack((ch_1_all, ch_1))
        ch_2_all = np.vstack((ch_2_all, ch_2))

    # plot data
    # fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    # ax.plot(df["timestamp"], df["differential_potential_CH1"], label="differential_potential_CH1")
    # ax.plot(df["timestamp"], df["differential_potential_CH2"], label="differential_potential_CH2")
    # fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    # ax.plot(interpolated_df[:, 0], label="differential_potential_CH1")
    # ax.plot(interpolated_df[:, 1], label="differential_potential_CH2")
    # ax.set_xlabel("Time")
    # ax.set_ylabel("Voltage [mV]")
    # ax.legend()
    # plt.show()

print(ch_1_all.shape)
print(ch_2_all.shape)

df = pd.DataFrame(ch_1_all)
df.to_csv(f"data/dataset/{stimulus}/ch_1.csv", index=False, header=False)
df = pd.DataFrame(ch_2_all)
df.to_csv(f"data/dataset/{stimulus}/ch_2.csv", index=False, header=False)



### Preparing the classification data set
Now, that we have features extracted, we need to prepare the data for classification by adding class information and splitting the data in training and test set.

#### Add class information
Here we structure the data into feature vectors and the corresponding class labels.

In [ ]:
wind_features = pd.read_csv("data/features/wind/mean_std_asp_features.csv")
temperature_features = pd.read_csv("data/features/temperature/mean_std_asp_features.csv")
# red_features = pd.read_csv("data/features/red_light/mean_std_asp_features.csv")
# blue_features = pd.read_csv("data/features/blue_light/mean_std_asp_features.csv")
#
# mean_wind = wind_features["mean_ch1"].tolist()
# mean_wind.extend(wind_features["mean_ch2"].tolist())
# std_wind = wind_features["std_ch1"].tolist()
# std_wind.extend(wind_features["std_ch2"].tolist())
# asp_wind = wind_features["asp_ch1"].tolist()
# asp_wind.extend(wind_features["asp_ch2"].tolist())
# wind_features = pd.DataFrame({"mean": mean_wind, "std": std_wind, "asp": asp_wind})
#
# mean_temperature = temperature_features["mean_ch1"].tolist()
# mean_temperature.extend(temperature_features["mean_ch2"].tolist())
# std_temperature = temperature_features["std_ch1"].tolist()
# std_temperature.extend(temperature_features["std_ch2"].tolist())
# asp_temperature = temperature_features["asp_ch1"].tolist()
# asp_temperature.extend(temperature_features["asp_ch2"].tolist())
# temperature_features = pd.DataFrame({"mean": mean_wind, "std": std_wind, "asp": asp_wind})

all_feature_names = wind_features.columns.to_list()
wind_features = wind_features.to_numpy()
temperature_features = temperature_features.to_numpy()

# shuffle data
np.random.seed(10)  # TODO set to reproduce results
np.random.shuffle(wind_features)
np.random.shuffle(temperature_features)

# select random features from wind
train_len = int(0.7 * wind_features.shape[0])
test_len = int(0.3 * wind_features.shape[0])
x_wind_train = wind_features[:train_len, :]
x_wind_test = wind_features[train_len:, :]
y_wind_train = np.zeros((x_wind_train.shape[0],))
y_wind_test = np.zeros((x_wind_test.shape[0],))


# select random features from temperature
train_len = int(0.7 * temperature_features.shape[0])
test_len = int(0.3 * temperature_features.shape[0])
x_temperature_train = temperature_features[:train_len, :]
x_temperature_test = temperature_features[train_len:, :]
y_temperature_train = np.ones((x_temperature_train.shape[0],))
y_temperature_test = np.ones((x_temperature_test.shape[0],))

# combine all classes
x_train = np.concatenate((x_wind_train, x_temperature_train), axis=0)
y_train = np.concatenate((y_wind_train, y_temperature_train), axis=0)
x_test = np.concatenate((x_wind_test, x_temperature_test), axis=0)
y_test = np.concatenate((y_wind_test, y_temperature_test), axis=0)


print(f"x_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"x_test: {x_test.shape}, y_test: {y_test.shape}")

## 4. Classification - Classical Machine Learning
In this section we will use the previous generated dataset for classification.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

number_of_features = len(all_feature_names)

# scaling the features based on the training set
min_max = MinMaxScaler()
for i in range(number_of_features):  # min max scale all the features
    x_train[:, i] = min_max.fit_transform(np.reshape(x_train[:, i], (x_train[:, i].shape[0], 1))).squeeze()
    x_test[:, i] = min_max.transform(np.reshape(x_test[:, i], (x_test[:, i].shape[0], 1))).squeeze()

# make classifier and fit
# estimator = SVC(kernel="poly", probability=True)  # , class_weight='balanced'
estimator = LinearDiscriminantAnalysis()
selector = estimator.fit(x_train, y_train)
y_pred = selector.predict(x_test)

## SHAP for explainabiltiy
# x_t = pd.DataFrame(data=x_test, columns=all_feature_names)
# x_train_wn = pd.DataFrame(data=x_train, columns=all_feature_names)
#
# explainer = shap.KernelExplainer(selector.predict_proba, x_train_wn)
# shap_value = explainer(x_t)
# # returns probability for class 0 and 1, but we only need one bc p = 1 - p
# shap_value.values = shap_value.values[:, :, 1]
# shap_value.base_values = shap_value.base_values[:, 1]
#
# print_frame = pd.DataFrame(data=np.zeros((1, number_of_features)), columns=all_feature_names)
# print_frame[:] = shap_value.abs.mean(axis=0).values
# print("----------")
# for z in print_frame.columns:
#     print(f"{z}\t{print_frame[z].to_numpy().squeeze()}")
#     # print(f"{print_frame[z].to_numpy().squeeze()}")
# print("----------")


print(f"accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1-score: {f1_score(y_test, y_pred, average='weighted')}")
print(f"confusion matrix: \n{confusion_matrix(y_test, y_pred)}")

## 5. Classification - Deep Learning
In this section we will use the previous generated dataset for classification using deep learning.

In [ ]:
n_classes = 2
np.random.seed(10)  # TODO set to reproduce results
split_ratio = 0.8
batch_size = 32
model_type = "LSTM"  # Dense or LSTM

wind_ch1 = genfromtxt("data/dataset/wind/ch_1.csv", delimiter=',')
wind_ch2 = genfromtxt("data/dataset/wind/ch_2.csv", delimiter=',')
temperature_ch1 = genfromtxt("data/dataset/temperature/ch_1.csv", delimiter=',')
temperature_ch2 = genfromtxt("data/dataset/temperature/ch_2.csv", delimiter=',')

print(wind_ch1.shape, wind_ch2.shape, temperature_ch1.shape, temperature_ch2.shape)

wind = np.concatenate((wind_ch1, wind_ch2), axis=0)
# shuffle the data
idx = np.random.permutation(wind.shape[0])
wind = wind[idx, :]

split_index = int(split_ratio * wind.shape[0])
wind_train = wind[:split_index, :]
wind_test = wind[split_index:, :]

temperature = np.concatenate((temperature_ch1, temperature_ch2), axis=0)
idx = np.random.permutation(temperature.shape[0])
temperature = temperature[idx, :]

split_index = int(split_ratio * temperature.shape[0])
temperature_train = temperature[:split_index, :]
temperature_test = temperature[split_index:, :]

x_train = np.concatenate((wind_train, temperature_train), axis=0)
y_train = np.concatenate((np.zeros((wind_train.shape[0],)), np.ones((temperature_train.shape[0],))))

x_test = np.concatenate((wind_test, temperature_test), axis=0)
y_test = np.concatenate((np.zeros((wind_test.shape[0],)), np.ones((temperature_test.shape[0],))))

# make dim right for LSTM
if model_type == "LSTM":
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# do one hot encoding to get the probability distribution
enc = preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
# You can blur the labels for better numerical stability
# y_train[y_train == 1] -= (0.01 * (n_classes-1))
# y_train[y_train == 0] += 0.01
#
# y_test[y_test == 1] -= (0.01 * (n_classes-1))
# y_test[y_test == 0] += 0.01

# Let's make our dataset performant using tf.data API
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# build model
model = Sequential()
if model_type == "LSTM":
    model.add(keras.layers.LSTM(100, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(keras.layers.LSTM(100))
    model.add(keras.layers.Flatten())
elif model_type == "Dense":
    model.add(InputLayer(input_shape=(x_train.shape[1],), batch_size=batch_size))
    # model.add(Dense(1000, activation='relu'))
    # model.add(Dense(720, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='sigmoid'))  # see: https://stackoverflow.com/questions/49016723/softmax-cross-entropy-loss-explodes
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

print(f"is gpu available: {tf.config.list_physical_devices('GPU')}")
print(model.summary())
# fit model
# save best model for later use
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"neural_network/best_{model_type}_model",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True)

history = model.fit(train_dataset, validation_data=test_dataset, epochs=100, batch_size=32,
                    callbacks=[model_checkpoint_callback], verbose=True)

# validate model -> use the best model
model = tf.keras.models.load_model(f"neural_network/best_{model_type}_model")
y_pred = model.predict(x_test, verbose=True)
keras.backend.clear_session()

# calculate metrics
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

acc = accuracy_score(y_test, y_pred)
f1_s = f1_score(y_test, y_pred, average='binary')  # average=None returns score for each class
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1 Score: {f1_s}")
print(f"Confusion matrix: \n{cm}")

# plotting the loss and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
xc = range(len(history.history['loss']))

plt.figure()
plt.plot(xc, train_loss, label="train loss")
plt.plot(xc, val_loss, label="val loss")
plt.plot(xc, train_acc, label="train accuracy")
plt.plot(xc, val_acc, label="val accuracy")
plt.title(f"{model_type} loss and accuracy")
plt.legend()
plt.show()
plt.savefig(f"plots/{model_type}_loss_acc_.pdf")


## 6. Challenge
Now that you have seen the basic approaches, you can try to implement your own classifier. The best performing classifier will win a prize. You can use all resources! Best of luck

In [ ]:
# TODO make the best classifier and win a prize